# Exercice 1

In [4]:
import gymnasium as gym
import numpy as np

#initialisation de l'environnement
env=gym.make("Taxi-v3")

#Nombre d'etats d'actions
state_size=env.observation_space.n
action_size=env.action_space.n

print(f"Nombre d'etats {state_size} Nombre d'actions {action_size}")

Nombre d'etats 500 Nombre d'actions 6


In [5]:
policy_table = np.ones((state_size, action_size)) / action_size  # Distribution uniforme


value_table = np.zeros(state_size)

print("\n📝 Premières lignes de la table de politique (policy_table) :")
print(policy_table[:5])  # Afficher les 5 premières lignes

print("\n📝 Premières lignes de la table de valeurs (value_table) :")
print(value_table[:5])  # Afficher les 5 premières valeurs


📝 Premières lignes de la table de politique (policy_table) :
[[0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
 [0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
 [0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
 [0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
 [0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]]

📝 Premières lignes de la table de valeurs (value_table) :
[0. 0. 0. 0. 0.]


# Exercice2 

In [6]:
state,_=env.reset()

for t in range(20):
    action=env.action_space.sample()
    observation,reward,done,_,_=env.step(action)
    print(f"Action :{action}  ,Observations:{observation}  ,Reward:{reward}")

    if done:
        env.reset()

env.close()

Action :0  ,Observations:223  ,Reward:-1
Action :4  ,Observations:223  ,Reward:-10
Action :3  ,Observations:203  ,Reward:-1
Action :2  ,Observations:223  ,Reward:-1
Action :0  ,Observations:323  ,Reward:-1
Action :3  ,Observations:323  ,Reward:-1
Action :0  ,Observations:423  ,Reward:-1
Action :0  ,Observations:423  ,Reward:-1
Action :5  ,Observations:423  ,Reward:-10
Action :5  ,Observations:423  ,Reward:-10
Action :1  ,Observations:323  ,Reward:-1
Action :2  ,Observations:343  ,Reward:-1
Action :0  ,Observations:443  ,Reward:-1
Action :4  ,Observations:443  ,Reward:-10
Action :3  ,Observations:423  ,Reward:-1
Action :0  ,Observations:423  ,Reward:-1
Action :0  ,Observations:423  ,Reward:-1
Action :5  ,Observations:423  ,Reward:-10
Action :2  ,Observations:443  ,Reward:-1
Action :0  ,Observations:443  ,Reward:-1


# Exercice 3

In [14]:
# Paramètres de PPO
gamma = 0.99  # Facteur d'escompte
learning_rate = 0.1
clip_epsilon = 0.2
num_iterations = 10  # Nombre de mises à jour de la politique

for _ in range(num_iterations):
    episode_states = []
    episode_actions = []
    episode_rewards = []

    # Collecte d'un épisode
    state,_ = env.reset()
    done = False
    while not done:
        action = np.random.choice(action_size, p=policy_table[state])  # Politique actuelle
        next_state, reward, terminated, _, _ = env.step(action)
        done = terminated 

        episode_states.append(state)
        episode_actions.append(action)
        episode_rewards.append(reward)

        state = next_state  # Mise à jour de l'état

    # Calcul des récompenses cumulées (discounted rewards)
    G = 0
    returns = []
    for reward in reversed(episode_rewards):
        G = reward + gamma * G
        returns.insert(0, G)

    # Normalisation des récompenses
    returns = np.array(returns)
    returns = (returns - np.mean(returns)) / (np.std(returns) + 1e-8)

    # Mise à jour de la table de valeur
    for state, G in zip(episode_states, returns):
        value_table[state] += learning_rate * (G - value_table[state])

    # Mise à jour de la politique avec PPO (clipping)
    for state, action, G in zip(episode_states, episode_actions, returns):
        advantage = G - value_table[state]
        old_prob = policy_table[state, action]

        # Calcul du ratio et application du clip
        new_prob = old_prob + learning_rate * advantage
        new_prob = np.clip(new_prob, old_prob * (1 - clip_epsilon), old_prob * (1 + clip_epsilon))

        # Normalisation de la politique
        policy_table[state] = np.clip(policy_table[state], 0, 1)
        policy_table[state] /= np.sum(policy_table[state])

print(f"La Policy Table \n {policy_table}")
print("---------------------------------------"*3)
print(f"La Value Table \n {value_table}")

La Policy Table 
 [[0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
 [0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
 [0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
 ...
 [0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
 [0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
 [0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]]
---------------------------------------------------------------------------------------------------------------------
La Value Table 
 [ 0.00000000e+00 -2.75975382e-01  2.42431978e-01  1.17810125e+00
  9.85021251e-02  0.00000000e+00  2.35985199e-01  2.08264397e-01
 -7.14184139e-02 -6.10646482e-01  0.00000000e+00  5.98811694e-01
  1.83506377e-01  2.35165793e-01  2.83396481e-01  0.00000000e+00
  4.83318533e+00  3.14355867e-01  2.40008931e+00  1.03560839e+00
  0.00000000e+00 -2.95328938e-01  1.64176868e-01  1.37331734e+00
 -1.89364422e-01  0.00000000e+00  3.4

# Exercice 4

In [16]:
num_eval_episodes = 5
total_rewards = []

for ep in range(num_eval_episodes):
    state, _ = env.reset()
    total_reward = 0
    done = False

    while not done:
        action = np.argmax(policy_table[state])  # Prendre la meilleure action selon la politique entraînée
        next_state, reward, terminated, _, _ = env.step(action)
        done = terminated 

        total_reward += reward
        state = next_state

    total_rewards.append(total_reward)
    print(f"Épisode {ep + 1}, Récompense totale : {total_reward}")

print(f"Récompense moyenne après entraînement : {np.mean(total_rewards)}")


KeyboardInterrupt: 